In [15]:
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import MobileNetV2

import cv2

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer

from imutils import paths
import numpy as np
import os

print('Libraries Loaded')

Libraries Loaded


In [6]:
path = os.getcwd()+'/Dataset'
images = list(paths.list_images(path))
data = []
labels = []

# loop over the image paths
for image in images:
    # extract the class label from the filename
    label = image.split(os.path.sep)[-2]
    
    # load the input image (224x224) and preprocess it
    loaded_image = keras.preprocessing.image.load_img(image, target_size=(224, 224))
    loaded_image = keras.preprocessing.image.img_to_array(loaded_image)
    loaded_image = keras.applications.mobilenet_v2.preprocess_input(loaded_image)
    
    # update the data and labels lists, respectively
    data.append(loaded_image)
    labels.append(label)

data = np.array(data, dtype="float32")
labels = np.array(labels)

print(f'The shape of the Data is: {data.shape}')
print(f'The shape of the labels is: {labels.shape}')

The shape of the Data is: (2757, 224, 224, 3)
The shape of the labels is: (2757,)


In [9]:
encoder = LabelBinarizer()
labels = encoder.fit_transform(labels)

In [12]:
labels_cat = to_categorical(labels)

In [30]:
(x_train, x_test, y_train, y_test) = train_test_split(data, labels_cat,test_size=0.20, random_state=42)
print(f'The shape of X_train is {x_train.shape} and the shape of X_test is {x_test.shape}')
print(f'The shape of Y_train is {y_train.shape} and the shape of Y_test is {y_test.shape}')

The shape of X_train is (2205, 224, 224, 3) and the shape of X_test is (552, 224, 224, 3)
The shape of Y_train is (2205, 2) and the shape of Y_test is (552, 2)


In [17]:
aug = keras.preprocessing.image.ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,
                                                   height_shift_range=0.2,shear_range=0.15,
                                                   horizontal_flip=True,fill_mode="nearest")

In [19]:
# Using pre-trained model
baseModel = MobileNetV2(weights="imagenet", include_top=False,input_tensor=keras.layers.Input(shape=(224, 224, 3)))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/Users/saad/opt/anaconda3/lib/python3.7/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


9412608/9406464 [==============================] - 3s 0us/step


In [22]:
# Remaining model
remainingModel = baseModel.output
remainingModel = keras.layers.AveragePooling2D(pool_size=(7, 7))(remainingModel)
remainingModel = keras.layers.Flatten(name="flatten")(remainingModel)
remainingModel = keras.layers.Dense(128, activation="relu")(remainingModel)
remainingModel = keras.layers.Dropout(0.5)(remainingModel)
remainingModel = keras.layers.Dense(2, activation="softmax")(remainingModel)

In [25]:
model = keras.Model(inputs=baseModel.input, outputs=remainingModel)

In [26]:
for layer in baseModel.layers:
    layer.trainable = False

In [27]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
______________________________________________________________________________________________

In [28]:
model.compile(loss="binary_crossentropy", optimizer='adam',metrics=["accuracy"])

In [ ]:
history = model.fit(aug.flow(x_train, y_train, batch_size=32),validation_data=(x_test, y_test),epochs=20)

Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
69/69 [==============================] - 84s 1s/step - loss: 0.4059 - acc: 0.8222 - val_loss: 0.1790 - val_acc: 0.9293
Epoch 2/20
69/69 [==============================] - 80s 1s/step - loss: 0.2444 - acc: 0.9098 - val_loss: 0.1416 - val_acc: 0.9493
Epoch 3/20
64/69 [==========================>...] - ETA: 0s - loss: 0.1464 - acc: 0.9062

In [2]:
face_detector = keras.Sequential()
face_detector.add(keras.layers.Conv2D(62,(3,3),activation='relu',input_shape=(150,150,3)))
face_detector.add(keras.layers.MaxPooling2D(2,2))
face_detector.add(keras.layers.Conv2D(62,(3,3),activation='relu'))
face_detector.add(keras.layers.MaxPooling2D(2,2))
face_detector.add(keras.layers.Flatten())
face_detector.add(keras.layers.Dropout(0.3))
face_detector.add(keras.layers.Dense(50,activation='relu'))
face_detector.add(keras.layers.Dense(2,activation='sigmoid'))

face_detector.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
TRAINING_DIR = os.getcwd()+"/Dataset/train"
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255,
                                                             rotation_range=40,
                                                             width_shift_range=0.2,
                                                             height_shift_range=0.2,
                                                             shear_range=0.2,
                                                             zoom_range=0.2,
                                                             horizontal_flip=True,
                                                             fill_mode='nearest',validation_split=0.2)

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, batch_size=10,target_size=(150, 150),subset='training')
validation_generator = train_datagen.flow_from_directory(TRAINING_DIR, batch_size=10,target_size=(150, 150),subset='validation')


TEST_DIR = os.getcwd()+"/Dataset/test"
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(TEST_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 1053 images belonging to 2 classes.
Found 262 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [4]:
checkpoint = keras.callbacks.ModelCheckpoint('Best_weights.hdf5',
                                             monitor='val_loss',
                                             verbose=0,
                                             save_best_only=True,
                                             mode='auto')

In [5]:
history = face_detector.fit_generator(train_generator,
                                      epochs=10,validation_data=test_generator,
                                      callbacks=[checkpoint])

Epoch 1/10
 40/106 [==========>...................] - ETA: 11s - loss: 0.7083 - acc: 0.5941

/Users/saad/opt/anaconda3/lib/python3.7/site-packages/PIL/Image.py:2817: UserWarning: image file could not be identified because WEBP support not installed
  warnings.warn(message)


 49/106 [============>.................] - ETA: 10s - loss: 0.6894 - acc: 0.6077

/Users/saad/opt/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


OSError: cannot identify image file <_io.BytesIO object at 0x7fc78332ae90>

In [6]:
wget 'https://1drv.ms/u/s!AskOIJpg2v0ZaAbMci-VUO9DC0o?e=WRGuLT'

SyntaxError: invalid syntax (<ipython-input-6-1085d61a3ec4>, line 1)